# Gmail API Credential Guide

Use Gmail API to Read Your Emails
1. Set up a Google Cloud Project

    Go to Google Cloud Platform(GCP) Console

    Create a new project

2. Enable the Gmail API for that project

    Select the created project and go to "APIs & services" page

    Click "+ Enable APIs and services" button, search "Gmail API" and enable it

3. Go to "OAuth Consent Screen" and configure:

    Choose External and Fill in app name, dedveloper email, etc.

4. Create OAuth Credentials

    Go to APIs & Services > Credentials

    Click "+ Create Credentials" > "OAuth client ID"

    Choose Desktop App

    Download the generated credentials.json

    Sometimes, GCP will navigate you to "Google Auth Platform" > "Clients", and you can click "+ Create client" here to create the OAuth Credentials

    
5. Add Test Users for Gmail API OAuth Access
    
    Go to "APIs & Services" > "OAuth consent screen" > "Audience" > "Test Users"

    Add the email account from which you want to extract email content.


6. Create 'credentials' folders to store gmail credential and user tokens

In [ ]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function
import os.path
import base64
import re
from email import message_from_bytes
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these SCOPES, delete the token.json
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
PORT = 18000

def main():
    creds = None
    # token.json stores the user's access and refresh tokens
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials/gmail_credentials.json', SCOPES)
        creds = flow.run_local_server(port=PORT)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)

    # Get the latest message
    results = service.users().messages().list(userId='me', maxResults=1).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    msg = service.users().messages().get(userId='me', id=messages[0]['id'], format='raw').execute()
    raw_msg = base64.urlsafe_b64decode(msg['raw'].encode('ASCII'))
    email_message = message_from_bytes(raw_msg)

    subject = email_message['Subject']
    print("Subject:", subject)

    # Extract text/plain body
    for part in email_message.walk():
        if part.get_content_type() == 'text/plain':
            print("Body:")
            print(part.get_payload(decode=True).decode('utf-8'))

if __name__ == '__main__':
    main()